In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# load the japanese phone dictionary 
def load_phone_dict(path):
  phone_dict = list()
  with open(path, 'r') as f:
    for line in f:
      phone_dict.append(line.strip())
  return phone_dict  

In [89]:
def phone_to_one_hot(phone, phone_dict):
  tmp = list(np.zeros(len(phone_dict)))
  if phone=='None':   
    return tmp
  else:
    tmp[phone_dict.index(phone)] = 1.0
    return tmp

In [90]:
def time_to_frame(time, window_size, step_size):
  number_frames = np.round(time/step_size)+1
  if number_frames >= 0:
    return number_frames
  else:
    return 0

In [142]:
#### convert /mono/.labs to csv
#### a floder under /mono/ called /csv/ will be created and it will hold all the csv files with current, next, before
#### the format of one example will be shown below

# label_path = '/Users/ShiyuMu/Desktop/HTS-demo_NIT-SONG070-F001/data/labels/mono/'
label_path = '/Users/ShiyuMu/Downloads/coldplay_dataset/txt/'
fileList = [label_path + f for f in os.listdir(label_path) if f.endswith('.txt')]
# fileList = [label_path + f for f in os.listdir(label_path) if f.endswith('.lab')]
try:
  # create dictionary to hold all the csv file 
  os.mkdir(label_path + 'csv/');
  print('csv dictionary created')
except:
  pass
for file in fileList:
  df = pd.read_csv(file, sep=" ", header=None)
  df.columns = ["beg", "end", "current"]
  df['before'] = pd.Series(['None']).append(df['current']).reset_index(drop=True)
  df['next'] = df['current'][1:].append(pd.Series(['None'])).reset_index(drop=True)
  df.to_csv(label_path + 'csv/'+ file.split('/')[-1]+'.csv', sep='\t', index=False)

In [143]:
fileList[0]

'/Users/ShiyuMu/Downloads/coldplay_dataset/txt/coldplay_song02_04.txt'

In [144]:
## load demision
dimension_dict = dict()
dimension_file = open('dimensions.txt', 'r')
for line in dimension_file:
  dimension_dict[line.strip().split()[0].strip('.npy')] = line.strip().split()[1]

In [145]:
# dimension_dict

### example of loading csv files 

In [146]:
## this is the path we just created
csv_path = label_path + 'csv/'
# get all csv files 
csvList = [csv_path + f for f in os.listdir(csv_path) if f.endswith('.csv')]
df_list = list()
## add all csv files as dataframe to a list
for file in csvList:
  df = pd.read_csv(file, sep='\t')
  df_list.append(df)

In [147]:
# csvList

In [148]:
## the df_list contains 31 dataframe (since we have 31 dataset)
## one dataframe will look like this. beg and end here indicates nonoseconds
second_to_nano = 10000000
# second_to_nano = 100000000
df_list[0].head()
for i in range(len(df_list)):
  
  df_list[i]['beg'] = df_list[i]['beg']*second_to_nano
  df_list[i]['end'] = df_list[i]['end']*second_to_nano
  df_list[i].beg = df_list[i].beg.astype(int)
  df_list[i].end = df_list[i].end.astype(int)


In [149]:
# clips_ids = ["004", "007", "037", "010", "004", "004", "004", "004"]
# durations = [(1.1, 2.2), (6.5, 7.8), (5.4, 6.8), (15.6, 16.8), (12.0, 13.2), (17.9, 19.0), (21.9, 22.7), (26.4, 27.2)]

In [150]:
df_list[10].tail()

,beg,end,current,before,next
14,18100000,20800000,er,d,n
15,20800000,22599999,n,er,iy
16,22599999,27599999,iy,n,th
17,27599999,27700000,th,iy,pau
18,27700000,29373242,pau,th,None


### load phoneme dictionary

In [151]:
# create phoneme dictionary list
dict_set = set()
for f in fileList:
  file = open(f, 'r')
  for line in file:
    dict_set.add(line.strip().split()[-1])
  file.close()
list(dict_set)
dict_file = open('english_dict.txt', 'w')
for phone in list(dict_set):
  dict_file.write(phone)
  dict_file.write('\n')
dict_file.close()

In [152]:
# len(list(dict_set))

In [153]:
phone_dict_path = 'english_dict.txt'
phone_dict = load_phone_dict(phone_dict_path)

### convert timestamps to frame numbers

In [172]:
#### define window size and step size to convert timestamp to frame id
window_size = 250000
step_size = 50000

for i in range(len(df_list)):
  df = df_list[i]
  df_index = i
#   print(csvList[df_index].split('/')[-1].rstrip('.txt.csv'))
  df_name = csvList[df_index].split('/')[-1].rstrip('.txt.csv')
  total_num_frames = int(dimension_dict[df_name])
#   print(total_num_frames)
  total_time = list(df['end'])[-1]
#   total_num_frames = time_to_frame(total_time, window_size, step_size)
#   print(len(df['beg'].values))
  for index in range(len(df['beg'].values)):
#     print(index)
    df['beg'].values[index] = np.floor(df['beg'].values[index]/total_time*total_num_frames)
#     df['beg'].values[index] = int(df['beg'].values[index]/total_time*total_num_frames)
#     df['beg'].values[index] = time_to_frame(df['beg'].values[index], window_size, step_size)
  for index in range(len(df['end'].values)):
#     print(index)
    df['end'].values[index] = np.floor(df['end'].values[index]/total_time*total_num_frames)
#     df['end'].values[index] = int(df['end'].values[index]/total_time*total_num_frames)

In [164]:
## now df_list contains 31 dataframe, one would be like this

df_list[-10]

,beg,end,current,before,next
0,0,148,pau,None,ay
1,148,236,ay,pau,pau
2,236,238,pau,ay,y
3,238,240,y,pau,uw
4,240,242,uw,y,z
5,242,262,z,uw,d
6,262,272,d,z,pau
7,272,274,pau,d,t
8,274,294,t,pau,uw
9,294,612,uw,t,pau


### Unfold all frames and list them one by one

In [131]:
def corase_encoding(current_index, beg, end):
  middle = int((end-beg)/2)
  if current_index == beg:
    return [1, 0, 0]
  elif current_index == middle:
    return [0, 1, 0]
  elif current_index == end:
    return [0, 0, 1]
  else:
    inverse_dist_beg = 1/np.abs(current_index-beg)
    inverse_dist_middle = 1/np.abs(current_index-middle)
    inverse_dist_end = 1/np.abs(current_index-end)
    p_beg = inverse_dist_beg/(inverse_dist_beg+inverse_dist_middle+inverse_dist_end)
    p_middle = inverse_dist_middle/(inverse_dist_beg+inverse_dist_middle+inverse_dist_end)
    p_end = inverse_dist_end/(inverse_dist_beg+inverse_dist_middle+inverse_dist_end)
    return [p_beg, p_middle, p_end]

In [173]:
unfolded_df_list = list()
for dataset in df_list:
  unfold_list = list()
  for index in range(dataset.shape[0]):
#     print(index)
    beg_ = int(dataset.iloc[index]['beg'] + 1)
    end_ = int(dataset.iloc[index]['end'])
#     print(end_)
    current_ = np.asarray(phone_to_one_hot(dataset.iloc[index]['current'], phone_dict))
    before_ = np.asarray(phone_to_one_hot(dataset.iloc[index]['before'], phone_dict))
    next_ = np.asarray(phone_to_one_hot(dataset.iloc[index]['next'], phone_dict))
    corase_encoding_vectors = np.asarray(corase_encoding(index, beg_, end_))
#     print(corase_encoding_vectors)


    if index == 0:
      for i in range(beg_, end_+1):  
        unfold_list.append([beg_, end_, np.asarray(phone_to_one_hot('None', phone_dict)), current_, next_, corase_encoding(i, beg_, end_)])
    elif index == (dataset.shape[0] -1):
      for i in range(beg_, end_+1):  
        unfold_list.append([beg_, end_, before_, current_, np.asarray(phone_to_one_hot('None', phone_dict)), corase_encoding(i, beg_, end_)])
    else:
      for i in range(beg_, end_+1):  
        unfold_list.append([beg_, end_, before_, current_, next_, corase_encoding(i, beg_, end_)])      
      
#     if index == 0:
#       unfold_list.append([beg_, end_, np.asarray(phone_to_one_hot('None', phone_dict)), current_, next_, corase_encoding(beg_, beg_, end_)])
#     else:
#       unfold_list.append([beg_, end_, before_, current_, next_, corase_encoding(beg_, beg_, end_)])
#       for i in range(beg_+1, end_):
#         unfold_list.append([beg_, end_, before_, current_, next_, corase_encoding(i, beg_, end_)])
#       if index == (dataset.shape[0] -1):
#         unfold_list.append([beg_, end_, before_, current_, np.asarray(phone_to_one_hot('None', phone_dict)), corase_encoding(end_, beg_, end_)])
#       else:
#         unfold_list.append([beg_, end_, before_, current_, next_, corase_encoding(end_, beg_, end_)])
      
  unfold_df = pd.DataFrame(unfold_list, columns=['beg', 'end', 'before', 'current', 'next', 'coarse'])
  unfolded_df_list.append(unfold_df)

In [174]:
unfolded_df_list[-10].shape

(3200, 6)

In [175]:
def creat_folder(folder_name):
  try:  
    os.mkdir(folder_name);
#     print('folder created: ', folder_name)
  except:
#     print('folder already exist: ', folder_name)
    pass

In [176]:
### Save npy files as required
creat_folder('npy')  
for i in range(len(csvList)):
  lab_name = csvList[i].split('/')[-1].split('.csv')[0]
  creat_folder('./npy/'+lab_name)
  print(lab_name.strip(".json.txt"))
#   creat_folder('./npy/'+lab_name+'/current')
#   creat_folder('./npy/'+lab_name+'/next')
#   creat_folder('./npy/'+lab_name+'/before')
#   creat_folder('./npy/'+lab_name+'/corase_encoding')
  print(unfolded_df_list[i]['current'].shape)
  np.save('./npy/'+lab_name+'/current.npy', unfolded_df_list[i]['current'])
  np.save('./npy/'+lab_name+'/next.npy', unfolded_df_list[i]['next'])
  np.save('./npy/'+lab_name+'/before.npy', unfolded_df_list[i]['before'])
  np.save('./npy/'+lab_name+'/corase_encoding.npy', unfolded_df_list[i]['coarse'])

coldplay_song06_04
(1047,)
coldplay_song07_02
(3930,)
coldplay_song05_01
(6123,)
coldplay_song01_06
(4232,)
coldplay_song03_04
(5963,)
coldplay_song06_05
(4436,)
coldplay_song02_02
(3567,)
coldplay_song07_03
(3997,)
coldplay_song05_02
(3124,)
coldplay_song01_05
(437,)
coldplay_song04_04
(588,)
coldplay_song06_07
(3402,)
coldplay_song03_06
(5859,)
coldplay_song07_01
(2591,)
coldplay_song06_06
(4324,)
coldplay_song02_01
(7053,)
coldplay_song01_04
(5510,)
coldplay_song04_05
(814,)
coldplay_song01_01
(3969,)
coldplay_song07_05
(947,)
coldplay_song02_04
(2207,)
coldplay_song06_13
(1072,)
coldplay_song06_03
(3576,)
coldplay_song03_02
(5983,)
coldplay_song07_04
(1462,)
coldplay_song02_05
(1028,)
coldplay_song06_02
(4785,)
coldplay_song06_12
(2692,)
coldplay_song04_01
(5964,)
coldplay_song03_01
(3200,)
coldplay_song06_10
(2929,)
coldplay_song04_03
(1333,)
coldplay_song01_02
(5983,)
coldplay_song06_09
(1845,)
coldplay_song04_02
(4789,)
coldplay_song01_03
(1571,)
coldplay_song06_08
(4299,)
coldp

In [ ]:
## Example of loading a noy file
np_array_example = np.load('./npy/nitech_jp_song070_f001_010.lab/current.npy')
np.array(np_array_example.tolist()).shape

## Since the requirement changed, the functions below are not useful anymore 

### save the unfolded dataframe to disk

In [ ]:
### a folder under /mono/csv/ called /unfolded/ will becreated which holds all datafames like above
try:
  # create dictionary to hold all the unfolded file 
  os.mkdir(label_path + 'csv/unfolded/');
  print('unfolded folder created')
except:
  print('folder already exist')
  pass
for i in range(len(csvList)):
  unfolded_df_list[i].to_csv(label_path + 'csv/unfolded/'+ csvList[i].split('/')[-1], sep='\t', index=False)

### After you created all the files, you can load them back to memory easily


## NOTICE, if you already have all the files saved, the code below is the only thing you need to load the file and start to work with your training. The code before is only for generating. 

In [ ]:
## this is the path we just created
unfolded_path = label_path + 'csv/unfolded/'
# get all csv files 
fileNameList = [unfolded_path + f for f in os.listdir(unfolded_path) if f.endswith('.csv')]
unfolded_df_list_ = list()
## add all csv files as dataframe to a list
for file in fileNameList:
  df = pd.read_csv(file, sep='\t')
  unfolded_df_list_.append(df)

In [ ]:
unfolded_df_list_[0].head()

### of course you may also want the filename of this specify dataframe to match with raw files

In [ ]:
# the index of fileNameList maintains the same with unfolded_df_list_
fileNameList[0].split('/')[-1]

### get one-hot-vector, call phone_to_one_hot() to a specific phoneme

In [ ]:
one_hot = phone_to_one_hot('sil', phone_dict)
print('sil', one_hot)

### to access one specifiy position

In [ ]:
### the first dataframe in the list, the index 0, and the current label
unfolded_df_list_[0].iloc[0]['current']